<a href="https://colab.research.google.com/github/Sulivart/Curso-Python-2023/blob/main/SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base de datos SQLite
<a href="https://colab.research.google.com/github/rambasnet/FDSPython-Notebooks/blob/master/Ch21-SqliteDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- https://www.sqlite.org/
- Basado en C, una de las bases de datos integradas más utilizadas (configuración cero)

## Conceptos básicos de SQL
- Lenguaje de consulta estructurado
- lenguaje insensible a mayúsculas y minúsculas; generalmente escrito en mayúsculas
- Permita que usted o el programa utilicen bases de datos basadas en SQL como SQLite, MySQL, MSSQL, PostgreSQL, etc.
- declaraciones básicas más importantes para aprender: CRUD
- C: crear (base de datos, tabla, crear e insertar registros)
- R: recuperar/leer datos
-U: actualizar datos
- D: eliminar datos
- http://www.w3schools.com/sql/default.asp

## Navegador SQLite
- Explorador de sqlite db basado en GUI
- facilita la visualización de datos y el aprendizaje de SQL
- http://sqlitebrowser.org/

# Módulo sqlite3
- python3 proporciona la biblioteca sqlite3 para trabajar con la base de datos sqlite
- https://docs.python.org/3/library/sqlite3.html
- SQLite admite de forma nativa los siguientes tipos: NULL, INTEGER, REAL, TEXT, BLOB

|SQLite type|Python type|
| ---|---|
| NULL | None |
| INTEGER | int |
| REAL | float |
| TEXT | str |
| BLOB | bytes |

## Ejemplo de base de datos en memoria

In [ ]:
import sqlite3
# Conectarse a la base de datos de la memoria
con = sqlite3.connect(":memory:")

# Crear una tabla
con.execute("create table person(fname, lname)")

In [ ]:
# Llenar la tabla con datos
persons = [('Hugo', 'Boss'), ('Calvin', 'Klien')]
con.executemany("insert into person(fname, lname) values (?, ?)", persons)

In [ ]:
# Imprimir el contenido de la tabla
for row in con.execute("select rowid, fname, lname from person"):
    print(row)

(1, 'Hugo', 'Boss')
(2, 'Calvin', 'Klien')


In [ ]:
print("I just deleted", con.execute("delete from person where rowid=1").rowcount, "rows")

I just deleted 1 rows


## Ejemplo de archivo db
### Crear base de datos, crear tabla e insertar datos en la tabla

In [ ]:
import sqlite3
# Creando conexión
conn = sqlite3.connect('example.db')
# Creando objeto de cursor
cur = conn.cursor()

cur.execute("""CREATE TABLE IF NOT EXISTS students (firstName text,
            lastName text, test1 real, test2 real, average real, grade text)""")

In [ ]:
query = """ INSERT INTO students (firstName, lastName, test1, test2) values (?, ?, ?, ?)"""
cur.execute(query, ('John', 'Smith', 99, 95.5))

In [ ]:
cur.execute(query, ('Michael', 'Jordan', 50, 65))

In [ ]:
# Guardar/confirmar los cambios en la base de datos
conn.commit()
# Cerrar la base de datos si está hecho
conn.close()

### Abrir base de datos, leer y actualizar tabla

In [ ]:
import sqlite3
conn = sqlite3.connect('example.db')
cur = conn.cursor()

In [ ]:
cur.execute('SELECT * FROM students where rowid = 1')
row = cur.fetchone() # Devuelve una fila como tupla si existe rowid con valor 1
print(row)

('John', 'Smith', 99.0, 95.5, None, None)


In [ ]:
for col in row:
    print(col)

John
Smith
99.0
95.5
None
None


In [ ]:
cur.execute('SELECT rowid, * FROM students')
rows = cur.fetchall()
print(rows)

[(1, 'John', 'Smith', 99.0, 95.5, None, None), (2, 'Michael', 'Jordan', 50.0, 65.0, None, None)]


In [ ]:
type(rows)

list

In [ ]:
for row in rows:
    print(row)

(1, 'John', 'Smith', 99.0, 95.5, None, None)
(2, 'Michael', 'Jordan', 50.0, 65.0, None, None)


<strong>Actualizar tabla</strong>

In [ ]:
for row in rows:
    avg = (row[3] + row[4])/2
    # grade = ?
    cur.execute('update students set average=? where rowid=?', (avg, row[0]))

In [ ]:
cur.execute('select * from students')
print(cur.fetchall())

[('John', 'Smith', 99.0, 95.5, 97.25, None), ('Michael', 'Jordan', 50.0, 65.0, 57.5, None)]


In [ ]:
# Confirmar cambios y cerrar conexión
conn.commit()
conn.close()

## Vulnerabilidad de inyección SQL
- Cómo no escribir consultas sql en programas

In [ ]:
import sqlite3
conn = sqlite3.connect('example.db')
cur = conn.cursor()

cur.execute("""CREATE TABLE IF NOT EXISTS users (username text unique, password text)""")

In [ ]:
# Solicitar al usuario que cree una cuenta
username = input('Enter your username: ')
password = input('Pick a password: ')

Enter your username: Sulivart
Pick a password: S2a9u0l7


In [ ]:
# Malas contraseñas
# Forma insegura de crear sentencias SQL
sqlinsert = "insert into users (username, password) values ('{0}', '{1}')".format(username, password)
print(sqlinsert)
cur.execute(sqlinsert)

insert into users (username, password) values ('Sulivart', 'S2a9u0l7')


In [ ]:
# Comprobar la base de datos
conn.commit()
for row in cur.execute('select * from users'):
    print(row)

('Sulivart', 'S2a9u0l7')


### ¿Qué hay de malo en los códigos anteriores?
### Autenticación de usuarios y ataque de inyección SQL

In [ ]:
# Solicitar al usuario que cree una cuenta
def insecureAuthentication():
    username = input('Enter your username: ')
    password = input('Pick a password: ')
    sqlSelect = "select * from users where username = '{0}' \
                    and password = '{1}'".format(username, password)
    cur.execute(sqlSelect)
    row = cur.fetchone()
    if row:
        print('Welcome {}, this is your kingdom!'.format(row[0]))
    else:
        print('Wrong credentials. Try Again!')

In [ ]:
insecureAuthentication()

Enter your username: Ody
Pick a password: 26072023
Wrong credentials. Try Again!


In [ ]:
# Inyección SQL: autenticación sin usar contraseña
insecureAuthentication()

Enter your username: Oda
Pick a password: 20230726
Wrong credentials. Try Again!


## Forma segura de almacenar la contraseña
- https://docs.python.org/3/library/hashlib.html

In [ ]:
import uuid
import hashlib, binascii

def createSecurePassword(password, salt=None, round=100000):
    if not salt:
        salt = uuid.uuid4().hex

    """
    for i in range(round):
        password = password+salt
        password = hashlib.sha256(password.encode('utf-8')).hexdigest()
    """
    # hashlib.pbkdf2_hmac(hash_name, password, salt, iterations, dklen=None)
    dk = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), salt.encode('utf-8'), round)
    password = binascii.hexlify(dk)
    return "%s:%s"%(password, salt)

In [ ]:
def secureRegistration():
    # Solicitar al usuario que cree una cuenta
    username = input('Enter your username: ')
    password = input('Enter your password: ')
    secPass = createSecurePassword(password)
    insert = 'insert into users (username, password) values (?, ?)'
    cur.execute(insert, (username, secPass))

In [ ]:
# Registrar un usuario
secureRegistration()

Enter your username: Coquetin
Enter your password: 29072003


In [ ]:
# Comprobar datos
for row in cur.execute('select * from users'):
    print(row)

('Sulivart', 'S2a9u0l7')
('Coquetin', "b'd15ce209705ef3e5ed9ffa89cd13bec377b7e1f2118e2af0c26be320faa3e5c7':267868e445f448ee98df67cd7e9c3a8c")


In [ ]:
conn.commit()

In [ ]:
def secureAuthentication():
    username = input('Enter your username: ')
    password = input('Enter your password: ')
    # use parameterized query
    sqlSelect = 'select password from users where username = ?'
    cur.execute(sqlSelect, (username,))
    row = cur.fetchone()
    if row:
        # username exists
        # check password hashes
        hashpass = row[0]
        hashedPass = hashpass[:hashpass.find(':')]
        salt = hashpass[hashpass.find(':')+1:]
        secPass = createSecurePassword(password, salt)
        if hashpass == secPass:
            print('Welcome to your kingdom, {}'.format(username))
        else:
            print('Wrong credentials. Try Again!')
    else:
        print('Wrong credentials. Try Again!')

In [ ]:
secureAuthentication()

Enter your username: MiPechocha
Enter your password: 29112001
Wrong credentials. Try Again!


In [ ]:
# Intente la misma inyección de SQL
secureAuthentication()

Enter your username: MiPechocha
Enter your password: 29112001
Wrong credentials. Try Again!


In [ ]:
conn.commit()
conn.close()